# Extraction of SDM
### Collect spin-density matrix elements from the `ROOT`-file and save them to a text file

In [ ]:
import ROOT
import os

In [ ]:
data_folder = '/Users/mikhailmikhasenko/cernbox/tmp/pwa_results'
sdm_folder = os.path.join('..', 'data', 'SDMs')

In [ ]:
name = '0.100000-0.112853'

In [ ]:
############################################

def get_cve(myfile, name, i) :
    hist = myfile.Get(name)
    return [hist.GetBinCenter(i), hist.GetBinContent(i), hist.GetBinError(i)]

############################################

def get_SDM(myfile,bin,nbins=88) :
    SDM = [[0.0 for x in range(nbins)] for y in range(nbins)]
    eSDM = [[0.0 for x in range(nbins)] for y in range(nbins)]
    # intensities
    for i in range(nbins) :
        hname = 'h'+str(i+1)
        if (myfile.GetListOfKeys().Contains(hname)) :
            v = get_cve(myfile,hname, bin)
            SDM[i][i] = v[1]
            eSDM[i][i] = v[2]
        else :
            print("Error: ",hname," not found!")
    # interferences
    for i in range(nbins) :
        for j in range(nbins) :
            if (i >= j) : continue
            rname = 'h1'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            iname = 'h2'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            if (myfile.GetListOfKeys().Contains(rname)) and (myfile.GetListOfKeys().Contains(iname)):
                vr = get_cve(myfile,rname, bin)
                vi = get_cve(myfile,iname, bin)
                SDM[i][j] = vr[1]+vi[1]*1j
                eSDM[i][j] = v[2]+vi[2]*1j
                SDM[j][i] = vr[1]-vi[1]*1j
                eSDM[j][i] = v[2]-vi[2]*1j
#             else :
#                 print("Error: ",rname," or ",iname," not found!")
    return SDM, eSDM
############################################

def save2file(output, inmat) :
    os.makedirs(os.path.dirname(output), exist_ok=True)
    # 
    mat=np.matrix([np.array(v).real for v in inmat])
    np.savetxt(output+'.re',mat)
    mat=np.matrix([np.array(v).imag for v in inmat])
    np.savetxt(output+'.im',mat)

def save_all_sdm(rootfilename, outfilename, Nb=100, Nw=88) :
    myfile = ROOT.TFile(rootfilename)
    for b in range(1,Nb+1) :
        SDM_b, eSDM_b = get_SDM(myfile,b,Nw)
        save2file(outfilename+'/'+'sdm'+str(b), SDM_b)
        save2file(outfilename+'/'+'sdm'+str(b)+'-err', eSDM_b)
    myfile.Close()    

## Check the functionality

In [ ]:
names = ["0.100000-0.112853", "0.112853-0.127471", "0.127471-0.144385", "0.144385-0.164401",
         "0.164401-0.188816", "0.188816-0.219907", "0.219907-0.262177", "0.262177-0.326380",
         "0.326380-0.448588", "0.448588-0.724294", "0.724294-1.000000"]

Save a single SDM

In [ ]:
myfile = ROOT.TFile(os.path.join(data_folder, 'hfit_'+names[2]+'.root'))

SDM91, eSDM91 = get_SDM(myfile,91)
print(SDM91[1][1]," (+-) ",eSDM91[1][1])

myfile.Close()

Save all SDMs for the fiven t' in the file 

In [ ]:
myfile = ROOT.TFile(os.path.join(data_folder, 'hfit_0.100000-0.112853.root'))

for b in range(1,101) :
    SDM_b, eSDM_b = get_SDM(myfile,b)
    save2file(os.path.join(sdm_folder, name, 'sdm'+str(b)), SDM_b)
    save2file(os.path.join(sdm_folder, name, 'sdm'+str(b)+'-err'), eSDM_b)
    
myfile.Close()

Save all SDMs form all t'-slices in files 

In [ ]:
for name in names :
    print(name)
    save_all_sdm(
        os.path.join(data_folder, 'hfit_'+name+'.root'),
        os.path.join(sdm_folder, name))

### Systematic uncertainties

Meanwhile, I did
```bash
cd /mnt/data/compass/2008/
# create folders
for i in $(ls flo_fit_results); do echo $i; mkdir flo_fit_results_txt/$i ; done 
# create subfolders with t-slices
for i in $(ls flo_fit_results); do echo "------> $i"; for l in $(ls flo_fit_results/$i); do N=$(echo $l | sed "s/hfit_//" | sed "s/.root//"); echo $N; mkdir flo_fit_results_txt/$i/$N; done; done
# copy foldes names
for i in $(ls flo_fit_results_txt/); do echo \"$i\",; done
# copy 22 t'-clices names
for i in $(ls flo_fit_results/flo_88waves_22tbins); do N=$(echo $i | sed "s/hfit_//" | sed "s/.root//"); echo \"$N\",; done
exit
```

In [ ]:
fold = "/mnt/data/compass/2008"
#
files=[("flo_88waves_best_11tbins",100,88),
    ("flo_53waves_best_11tbins",100,53),
#     ("flo_88waves_22tbins",100,88), # that is special (see below)
    ("flo_88waves_coarse_ES",100,88),
    ("flo_88waves_f0_980_BW",100,88),
    ("flo_88waves_K1",100,88),
    ("flo_88waves_no_neg_waves",100,81)#,
    ("flo_88waves_rank2",100,88)]

In [ ]:
for ff in files :
    fl = ff[0]
    print("--->",fl)
    root_folder = fold+"/flo_fit_results/"    +fl+"/"
    text_folder = fold+"/flo_fit_results_txt/"+fl+"/"
    for name in names :
        print(name)
        save_all_sdm(root_folder+'hfit_'+name+'.root',
                     text_folder+name, ff[1], ff[2])

### Special file with 22 $t'$ bins

In [ ]:
for ff in [("flo_88waves_22tbins",100,88)] :
    fl = ff[0]
    print("--->",fl)
    root_folder = fold+"/flo_fit_results/"    +fl+"/"
    text_folder = fold+"/flo_fit_results_txt/"+fl+"/"
    for name in ["0.100000-0.106427",
        "0.106427-0.112853",
        "0.112853-0.120162",
        "0.120162-0.127471",
        "0.127471-0.135928",
        "0.135928-0.144385",
        "0.144385-0.154399",
        "0.154399-0.164401",
        "0.164401-0.176609",
        "0.176609-0.188816",
        "0.188816-0.204362",
        "0.204362-0.219907",
        "0.219907-0.241042",
        "0.241042-0.262177",
        "0.262177-0.294228",
        "0.294228-0.326380",
        "0.326380-0.387484",
        "0.387484-0.448588",
        "0.448588-0.586441",
        "0.586441-0.724294",
        "0.724294-0.862147",
        "0.862147-1.000000"]:
        print(name)
        save_all_sdm(root_folder+'hfit_'+name+'.root',
                     text_folder+name, ff[1], ff[2])